In [1]:
import pandas as pd
from TextHandler import *

In [2]:
sources = pd.read_json('datasets/sources.json', orient='index')

df_credible_1 = pd.read_json('datasets/source_3/scraped_articles.json')
df_credible_2 = pd.read_json('datasets/source_6/scraped_articles.json')

df_not_credible_1 = pd.read_json('datasets/source_7/scraped_articles.json')
df_not_credible_2 = pd.read_json('datasets/source_14/scraped_articles.json')

print("data_Set_1_credible:",df_credible_1.count())
print("data_Set_2_credible:",df_credible_2.count())
print("data_Set_3_not-credible:",df_not_credible_1.count())
print("data_Set_4_not-credible:",df_not_credible_2.count())

data_Set_1_credible: articles    4394
dtype: int64
data_Set_2_credible: articles    1211
dtype: int64
data_Set_3_not-credible: articles    2704
dtype: int64
data_Set_4_not-credible: articles    3381
dtype: int64


In [3]:
df_credible_3 = pd.read_json('datasets/source_1/scraped_articles.json')

df_not_credible_3 = pd.read_json('datasets/source_8/scraped_articles.json')

print("data_Set_3_credible:",df_credible_3.count())
print("data_Set_3_not-credible:",df_not_credible_3.count())


data_Set_3_credible: articles    2288
dtype: int64
data_Set_3_not-credible: articles    3654
dtype: int64


In [4]:
df_credible_1['credibility'] = 1
df_credible_2['credibility'] = 1
df_not_credible_1['credibility'] = 0
df_not_credible_2['credibility'] = 0
df_credible_3['credibility'] = 1
df_not_credible_3['credibility'] = 0

merged_df = pd.concat([df_credible_1, df_credible_2, df_not_credible_1, df_not_credible_2,df_credible_3, df_not_credible_3])

# merged_df['credibility'] = 1
# merged_df.loc[merged_df.index.isin(df_not_credible_1.index) | merged_df.index.isin(df_not_credible_2.index), 'credibility'] = 0


texts = []
for article in merged_df['articles']:
    text = article['text']
    texts.append(text)

merged_df['text'] = texts

new_df = merged_df[['text', 'credibility']]
print("New DataFrame:")
print(new_df.head())

New DataFrame:
                                                text  credibility
0  تمكنت عناصر جهاز خفر السواحل الليبي، فجر اليوم...            1
1  أكد المبعوث الأميركي الخاص إلى السودان وجنوب ا...            1
2  ‬اطلع رئيس لجنة الإدارة المكلف بشركة الخليج ال...            1
3  ‬نشرت الشركة الليبية للموانئ بيانات حديثة أظهر...            1
4  طالب حراك “الاستفتاء أولا” بعرض مسودة الدستور ...            1


In [ ]:
# df_credible_3['credibility'] = 1
# df_not_credible_3['credibility'] = 0

# merged_df_test = pd.concat([df_credible_3, df_not_credible_3])

# print(merged_df_test['credibility'].value_counts()[1])

# texts = []
# for article in merged_df_test['articles']:
#     text = article['text']
#     texts.append(text)

# merged_df_test['text'] = texts

# new_df_test = merged_df_test[['text', 'credibility']]
# print("New DataFrame:")
# print(new_df_test.head())

# data_test=new_df_test['text']
# preprocessed_data_test=preprocess_text(data_test)
# print(preprocessed_data_test[0])

0    9739
1    7893
Name: credibility, dtype: int64


In [5]:
data=new_df['text']
preprocessed_data=preprocess_text(data)
print(preprocessed_data[0])

تمك عنصر جهز خفر سحل ليب فجر اليوم قاذ هجر طرق الا شواطاء الا ورب علا متن زورق ططي نقل وكل ليب نطق رئس ارك قوت بحر ان زورق زوي تحر تلق ندء غاث عمل علا قاذ هجر زال قعد طرابلس بحر نقل الا جهز كفح هجر قنن تمم جرء رحل بلد وكان خفر سحل ليب اعد ايم قلل هجر الا ليب خلل عمل قاذ نفذ طلع اعل تحدث بسم ظمه هجر دول


In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

from sklearn.model_selection import GridSearchCV

X_train, X_test, y_train, y_test = train_test_split(preprocessed_data, new_df['credibility'], test_size=0.2, random_state=42)

ngram_ranges = [(1, 1), (2, 2), (3, 3), (4, 4)]

classifiers = [
    LogisticRegression(),
    MultinomialNB(),
    KNeighborsClassifier(),
    SVC()
]


for classifier in classifiers:
    classifier_name = classifier.__class__.__name__
    print(f"Classifier: {classifier_name}")
    for ngram_range in ngram_ranges:
        vectorizer = TfidfVectorizer(ngram_range=ngram_range)
        X_train_tfidf = vectorizer.fit_transform(X_train)
        X_test_tfidf = vectorizer.transform(X_test)
        X_train_normalized = preprocessing.normalize(X_train_tfidf)

        classifier.fit(X_train_normalized, y_train)

        y_pred = classifier.predict(X_test_tfidf)

        accuracy = accuracy_score(y_test, y_pred)
        print(f"Accuracy for n-gram range {ngram_range}: {accuracy}")

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(preprocessed_data, new_df['credibility'], test_size=0.2, random_state=42)

ngram_ranges = [(1, 1), (2, 2), (3, 3), (4, 4)]

classifiers = [
    LogisticRegression(),
    MultinomialNB(),
    KNeighborsClassifier(),
    SVC()
]

for classifier in classifiers:
    classifier_name = classifier.__class__.__name__
    print(f"Classifier: {classifier_name}")
    for ngram_range in ngram_ranges:
        vectorizer = CountVectorizer(ngram_range=ngram_range)
        X_train_tfidf = vectorizer.fit_transform(X_train)
        X_test_tfidf = vectorizer.transform(X_test)
        X_train_normalized = preprocessing.normalize(X_train_tfidf)

        classifier.fit(X_train_normalized, y_train)

        y_pred = classifier.predict(X_test_tfidf)

        accuracy = accuracy_score(y_test, y_pred)
        print(f"Accuracy for n-gram range {ngram_range}: {accuracy}")

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold
from sklearn import preprocessing
import numpy as np

X = preprocessed_data
y = new_df['credibility']

ngram_ranges = [(1, 1), (2, 2), (3, 3), (4, 4)]

classifiers = [
    LogisticRegression(),
    MultinomialNB(),
    KNeighborsClassifier(),
    SVC()
]

kf = KFold(n_splits=5, shuffle=True, random_state=42)

for classifier in classifiers:
    classifier_name = classifier.__class__.__name__
    print(f"Classifier: {classifier_name}")
    for ngram_range in ngram_ranges:
        vectorizer = TfidfVectorizer(ngram_range=ngram_range)
        X_tfidf = vectorizer.fit_transform(X)

        accuracies = []
        for train_index, test_index in kf.split(X_tfidf):
            X_train, X_test = X_tfidf[train_index], X_tfidf[test_index]
            y_train, y_test = y[train_index], y[test_index]

            X_train_normalized = preprocessing.normalize(X_train)
            X_test_normalized = preprocessing.normalize(X_test)

            classifier.fit(X_train_normalized, y_train)
            y_pred = classifier.predict(X_test_normalized)
            accuracy = accuracy_score(y_test, y_pred)
            accuracies.append(accuracy)
        
        mean_accuracy = np.mean(accuracies)
        print(f"Accuracy for n-gram range {ngram_range}: {mean_accuracy}")


In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import KFold
from sklearn import preprocessing
import numpy as np


X = preprocessed_data
y = new_df['credibility']

ngram_ranges = [(1, 1), (2, 2), (3, 3), (4, 4)]

classifiers = [
    LogisticRegression(),
    MultinomialNB(),
    KNeighborsClassifier(),
    SVC()
]

kf = KFold(n_splits=5, shuffle=True, random_state=42)

for classifier in classifiers:
    classifier_name = classifier.__class__.__name__
    print(f"Classifier: {classifier_name}")
    for ngram_range in ngram_ranges:
        vectorizer = CountVectorizer(ngram_range=ngram_range)
        X_cv = vectorizer.fit_transform(X)

        accuracies = []
        for train_index, test_index in kf.split(X_cv):
            X_train, X_test = X_cv[train_index], X_cv[test_index]
            y_train, y_test = y[train_index], y[test_index]

            X_train_normalized = preprocessing.normalize(X_train)
            X_test_normalized = preprocessing.normalize(X_test)

            classifier.fit(X_train_normalized, y_train)
            y_pred = classifier.predict(X_test_normalized)
            accuracy = accuracy_score(y_test, y_pred)
            accuracies.append(accuracy)
        
        mean_accuracy = np.mean(accuracies)
        print(f"Accuracy for n-gram range {ngram_range}: {mean_accuracy}")


Classifier: LogisticRegression


KeyError: '[4394, 4395, 4396, 4398, 4399, 4402, 4403, 4404, 4405, 4406, 4408, 4409, 4411, 4412, 4414, 4415, 4416, 4417, 4418, 4419, 4420, 4421, 4422, 4423, 4424, 4425, 4426, 4427, 4428, 4429, 4430, 4431, 4432, 4433, 4434, 4435, 4437, 4439, 4440, 4442, 4443, 4444, 4445, 4446, 4448, 4451, 4452, 4453, 4454, 4455, 4456, 4457, 4458, 4460, 4462, 4463, 4465, 4466, 4467, 4468, 4470, 4471, 4472, 4473, 4474, 4476, 4477, 4478, 4479, 4480, 4481, 4482, 4483, 4484, 4485, 4486, 4487, 4489, 4490, 4491, 4492, 4493, 4494, 4495, 4496, 4497, 4498, 4499, 4500, 4503, 4504, 4505, 4507, 4508, 4511, 4512, 4513, 4514, 4515, 4516, 4517, 4520, 4521, 4522, 4523, 4524, 4526, 4529, 4530, 4531, 4532, 4534, 4535, 4536, 4537, 4538, 4539, 4540, 4541, 4542, 4543, 4544, 4545, 4549, 4550, 4551, 4552, 4555, 4556, 4557, 4558, 4559, 4562, 4563, 4565, 4566, 4567, 4568, 4569, 4570, 4571, 4573, 4574, 4575, 4576, 4577, 4578, 4579, 4581, 4582, 4583, 4584, 4585, 4586, 4587, 4588, 4589, 4590, 4591, 4592, 4593, 4594, 4595, 4596, 4597

In [33]:
import torch
import scipy.sparse
from torch import nn
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import numpy as np
from sklearn import preprocessing
from sklearn.utils import shuffle
from tqdm import tqdm
#clear pc ram of this variable if it exists
try:
    del X_tfidf
    del X_train_torch
    del X_test_torch
    del y_train_torch
    del logps
    del criterion
    del optimizer
    del model
except:
    pass



vectorizer = TfidfVectorizer(ngram_range=(2,2))

X = preprocessed_data
y = new_df['credibility']

#print unique labels and their counts
print("Unique labels and their counts: ")
print(np.unique(y, return_counts=True))

#shuffle the data
X, y = shuffle(X, y, random_state=42)
X_tfidf = vectorizer.fit_transform(X)
#float32
X_tfidf = X_tfidf.astype(np.float32)
accuracies = []

#cut down the data to 1/5 its size
denom = 7
X_tfidf = X_tfidf[:int(X_tfidf.shape[0] / denom)]    
y = y[:int(y.shape[0] / denom)]
# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=69)
print(X_tfidf.shape)
print(X_train.shape)

X_train_normalized = preprocessing.normalize(X_train)
X_test_normalized = preprocessing.normalize(X_test)

X_train_torch = torch.tensor(X_train_normalized.todense())
X_test_torch = torch.tensor(X_test_normalized.todense())
y_train_torch= torch.tensor(y_train.values)
y_test_torch = torch.tensor(y_test.values)

# model = nn.Sequential(
# nn.Linear(X_train_torch.shape[1], 64),
# nn.ReLU(),
# nn.Linear(64, [0,1]),
# nn.LogSoftmax(dim=1)
# )

# a better model that fixes the problem where the model stops learning after a few epochs
model = nn.Sequential(
nn.Linear(X_train_torch.shape[1], 12),
nn.ReLU(),
nn.Linear(12, X_train_torch.shape[1]),
nn.ReLU(),
nn.Linear(X_train_torch.shape[1], 2),
nn.Sigmoid()
)

criterion = nn.NLLLoss()
logps = model(X_train_torch)

loss = criterion(logps, y_train_torch)
loss.backward()
optimizer = torch.optim.Adam(model.parameters(), lr=0.002)

epochs = 30
for e in tqdm(range(epochs)):
    optimizer.zero_grad()
    output = model.forward(X_train_torch)
    loss = criterion(output, y_train_torch)
    loss.backward()
    optimizer.step()

    with torch.no_grad():
        output = model.forward(X_test_torch)
        ps = torch.exp(output)
        top_p, top_class = ps.topk(1, dim=1)
        accuracy = accuracy_score(y_test_torch, top_class)
        accuracies.append(accuracy)
    

# # use half the elements of the sparse matrix
# X_train_tfidf = scipy.sparse.csr_matrix(X_train_tfidf[:int(X_train_tfidf.shape[0] / 3)])
# X_test_tfidf = scipy.sparse.csr_matrix(X_test_tfidf[:int(X_test_tfidf.shape[0] / 3)])
# y_train = y_train[:int(y_train.shape[0] / 3)]
# y_test = y_test[:int(y_test.shape[0] / 3)]

# # check all unique text values in y_train and y_test, and assign numbers for each unique value in y_train and y_test, create a dictionary for this, then replace the values in y_train and y_test with the numbers, make sure to store the dictionary in a variable
# y_train = pd.DataFrame(y_train)
# y_test = pd.DataFrame(y_test)
# y_train['Category'] = y_train['Category'].astype('category')
# y_test['Category'] = y_test['Category'].astype('category')
# y_train_dict = dict(enumerate(y_train['Category'].cat.categories))
# y_test_dict = dict(enumerate(y_test['Category'].cat.categories))
# y_train['Category'] = y_train['Category'].cat.codes
# y_test['Category'] = y_test['Category'].cat.codes

# # convert to numpy arrays
# y_train = y_train.to_numpy()
# y_test = y_test.to_numpy()





# print(y_train)
# # convert to torch tensors
# X_train_torch = torch.from_numpy(X_train_tfidf.todense())
# X_test_torch = torch.from_numpy(X_test_tfidf.todense())
# #convert y to torch tensor (y is text)
# y_train_torch = torch.from_numpy(y_train)
# y_test_torch = torch.from_numpy(y_test)


# model = nn.Sequential(
#     nn.Linear(X_train_torch.shape[1], 64),
#     nn.ReLU(),
#     nn.Linear(64, df_2['Category'].nunique()),
#     nn.LogSoftmax(dim=1)
# )

# criterion = nn.NLLLoss()
# logps = model(X_train_torch)

# loss = criterion(logps, y_train_torch)
# loss.backward()
# optimizer = torch.optim.Adam(model.parameters(), lr=0.002)

# epochs = 50
# for e in range(epochs):
#     optimizer.zero_grad()
#     output = model.forward(X_train_torch)
#     loss = criterion(output, y_train_torch)
#     loss.backward()
#     optimizer.step()

#     with torch.no_grad():
#         model.eval()
#         log_ps = model(X_test_torch)
#         test_loss = criterion(log_ps, y_test_torch)
#         ps = torch.exp(log_ps)
#         top_p, top_class = ps.topk(1, dim=1)
#         equals = top_class == y_test_torch.view(*top_class.shape)
#         test_accuracy = torch.mean(equals.float())

Unique labels and their counts: 
(array([0, 1], dtype=int64), array([9739, 7893], dtype=int64))


C:\Users\Amr\AppData\Local\Temp\ipykernel_21736\4019258356.py:46: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  y = y[:int(y.shape[0] / denom)]


(2518, 860404)
(2014, 860404)


RuntimeError: [enforce fail at ..\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 6931414624 bytes.

In [ ]:
print(accuracies)
#Number of accuracies
print(len(accuracies))

[0.5714285714285714, 0.5992063492063492, 0.6488095238095238, 0.6686507936507936, 0.6884920634920635, 0.7083333333333334, 0.7202380952380952, 0.7420634920634921, 0.7638888888888888, 0.7837301587301587, 0.7936507936507936, 0.7896825396825397, 0.7956349206349206, 0.7976190476190477, 0.7956349206349206, 0.7996031746031746, 0.7976190476190477, 0.7996031746031746, 0.8035714285714286, 0.8075396825396826, 0.8115079365079365, 0.8154761904761905, 0.8174603174603174, 0.8174603174603174, 0.8214285714285714, 0.8194444444444444, 0.8174603174603174, 0.8174603174603174, 0.8174603174603174, 0.8174603174603174, 0.8174603174603174, 0.8174603174603174, 0.8174603174603174, 0.8174603174603174, 0.8174603174603174, 0.8174603174603174, 0.8174603174603174, 0.8174603174603174, 0.8134920634920635, 0.8134920634920635, 0.8134920634920635, 0.8134920634920635, 0.8134920634920635, 0.8134920634920635, 0.8134920634920635, 0.8134920634920635, 0.8134920634920635, 0.8115079365079365, 0.8134920634920635, 0.8134920634920635]